### Red Agents

Scenario 1b has two rules-based Red Agents. The first is our good friend the B_lineAgent. This represents an actor who has inside information, so is able to beeline straight towards the OpServer.

In [2]:
import inspect
from pprint import pprint
from CybORG import CybORG
from CybORG.Agents import *
from CybORG.Shared.Actions import *

path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario2.yaml'

env = CybORG(path,'sim')

agent = B_lineAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(16):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    print(action)

DiscoverRemoteSystems 10.0.154.144/28
DiscoverNetworkServices 10.0.154.155
ExploitRemoteService 10.0.154.155
PrivilegeEscalate User3
DiscoverNetworkServices 10.0.209.213
ExploitRemoteService 10.0.209.213
PrivilegeEscalate Enterprise0
DiscoverRemoteSystems 10.0.209.208/28
DiscoverNetworkServices 10.0.209.218
ExploitRemoteService 10.0.209.218
PrivilegeEscalate Enterprise2
DiscoverNetworkServices 10.0.21.142
ExploitRemoteService 10.0.21.142
PrivilegeEscalate Op_Server0
Impact Op_Server0
Impact Op_Server0


This agent runs along a predetermined path to the Op_Server, but is smart enough able to recover its position if interrupted. We can see below after Blue Team restores some hosts, the agent works out where the error in and re-exploits its way to the Op_Server.

In [3]:
action = Restore(hostname='Op_Server0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise2',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise1',session=0,agent='Blue')
env.step(action=action,agent='Blue')

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
            
    print(action)
    print('Success:',obs['success'])

Impact Op_Server0
Success: FALSE
PrivilegeEscalate Op_Server0
Success: FALSE
ExploitRemoteService 10.0.21.142
Success: TRUE
PrivilegeEscalate Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE
Impact Op_Server0
Success: TRUE


The other red agent is the MeanderAgent. This performs a breadth first search on all known hosts, scanning each one in turn, before attempting a mix of exploit and privilege escalate on the rest. This is an extremely slow agent in contrast to the laser-focussed B_lineAgent.

In [5]:
agent = RedMeanderAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(46):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    print(results.done)
    print(action)

False
DiscoverRemoteSystems 10.0.22.64/28
False
DiscoverNetworkServices 10.0.22.74
False
DiscoverNetworkServices 10.0.22.70
False
DiscoverNetworkServices 10.0.22.65
False
DiscoverNetworkServices 10.0.22.72
False
DiscoverNetworkServices 10.0.22.73
False
PrivilegeEscalate User0
False
ExploitRemoteService 10.0.22.74
False
ExploitRemoteService 10.0.22.73
False
PrivilegeEscalate User2
False
DiscoverNetworkServices 10.0.156.124
False
ExploitRemoteService 10.0.22.65
False
PrivilegeEscalate User4
False
DiscoverNetworkServices 10.0.156.115
False
ExploitRemoteService 10.0.156.124
False
PrivilegeEscalate Enterprise1
False
DiscoverRemoteSystems 10.0.156.112/28
False
DiscoverNetworkServices 10.0.156.116
False
DiscoverNetworkServices 10.0.156.122
False
ExploitRemoteService 10.0.156.116
False
ExploitRemoteService 10.0.22.70
False
PrivilegeEscalate User3
False
ExploitRemoteService 10.0.156.124
False
PrivilegeEscalate Enterprise1
False
ExploitRemoteService 10.0.22.72
False
PrivilegeEscalate User1
False

The Meander Agent is also able to recover from Blue's disruption.

In [6]:
action = Restore(hostname='Op_Server0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise2',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise1',session=0,agent='Blue')
env.step(action=action,agent='Blue')

action = Restore(hostname='Enterprise0',session=0,agent='Blue')
env.step(action=action,agent='Blue')

for i in range(24):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    print(env.get_last_action('Red'))

Impact Op_Server0
DiscoverRemoteSystems 10.0.20.112/28
ExploitRemoteService 10.0.150.26
ExploitRemoteService 10.0.150.29
PrivilegeEscalate Enterprise2
ExploitRemoteService 10.0.20.122
PrivilegeEscalate Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0
Impact Op_Server0


### Blue Agents

The BlueReactRemoveAgent will wait until it sees suspicious activity, before using remove on all the hosts it has flagged. However, due to the 5% change that Red's exploit is missed, Red will always eventually get to the Op_Server.

In [7]:
env = CybORG(path,'sim',agents={'Red':B_lineAgent})

agent = BlueReactRemoveAgent()

results = env.reset('Blue')
obs = results.observation
action_space = results.action_space

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    print(env.get_last_action('Blue'))

Monitor
Monitor
Monitor
Remove User2
Monitor
Monitor
Remove Enterprise1
Monitor
Remove Enterprise1
Monitor
Remove Enterprise1
Monitor


The BlueReactRestoreAgent is the same as the React agent above, but uses the Restore action.

In [8]:
agent = BlueReactRestoreAgent()

results = env.reset('Blue')
obs = results.observation
action_space = results.action_space

for i in range(12):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    print(env.get_last_action('Blue'))

Monitor
Monitor
Monitor
Restore User4
Monitor
Restore User4
Monitor
Restore User4
Monitor
Restore User4
Monitor
Restore User4


### Green Agent

An important part of CybORG Scenario1b is the Green agent, which represents the users on the network. The Green Agent is very simple, it only performs a scanning action on random hosts some of the time. This is only visible by Blue Agent.

In [9]:
agent = GreenAgent()

results = env.reset('Green')
obs = results.observation
action_space = results.action_space

for i in range(12):
    print(agent.get_action(obs,action_space))

Sleep
GreenPortScan
Sleep
Sleep
Sleep
Sleep
GreenPortScan
GreenPortScan
Sleep
GreenPortScan
GreenPortScan
GreenPortScan


### Keyboard Agent

The KeyboardAgent allows a human user to manually choose actions. This is useful for getting an intuition for the scenario.

In [ ]:
from CybORG.Agents.Wrappers import RedTableWrapper

cyborg = CybORG(path, 'sim',agents={'Blue':BlueMonitorAgent})
env = RedTableWrapper(env=cyborg, output_mode='table')

agent = KeyboardAgent()

results = env.reset('Red')
obs = results.observation
action_space = results.action_space

for i in range(3):
    print(obs)
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation

+-----------------+--------------+----------+---------+------------+
|      Subnet     |  IP Address  | Hostname | Scanned |   Access   |
+-----------------+--------------+----------+---------+------------+
| 10.0.103.224/28 | 10.0.103.230 |  User0   |  False  | Privileged |
+-----------------+--------------+----------+---------+------------+

************************************ Turn 1: Observation *************************************

+-----------------+--------------+----------+---------+------------+
|      Subnet     |  IP Address  | Hostname | Scanned |   Access   |
+-----------------+--------------+----------+---------+------------+
| 10.0.103.224/28 | 10.0.103.230 |  User0   |  False  | Privileged |
+-----------------+--------------+----------+---------+------------+

********************************* Turn 1: Command Selection **********************************

0 Sleep
1 DiscoverRemoteSystems
2 DiscoverNetworkServices
3 ExploitRemoteService
4 BlueKeep
5 EternalBlue
6 FTPDirec